## Imports

In [ ]:
#!pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.0
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../d1/datasets/titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum() / len(df)

Survived    0.000000
Pclass      0.000000
Name        0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Ticket      0.000000
Fare        0.000000
Cabin       0.771044
Embarked    0.002245
dtype: float64

In [5]:
df_test.isna().sum() / len(df_test)

Pclass      0.000000
Name        0.000000
Sex         0.000000
Age         0.205742
SibSp       0.000000
Parch       0.000000
Ticket      0.000000
Fare        0.002392
Cabin       0.782297
Embarked    0.000000
dtype: float64

# Exercise 1:
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
def get_title(string):
    string_ret = string.split(", ")[1].split(".")[0]
    return string_ret

df["Title"] = df['Name'].map(get_title)
df_test["Title"] = df_test['Name'].map(get_title)
df["Title"].unique()
df_test["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Col', 'Rev', 'Dr', 'Dona'],
      dtype=object)

# Exercise 2:
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [7]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [8]:
def group_titles(dataframe, column, dictionary):
    title_list = []
    for title in dataframe[column]:
        for k,v in dictionary.items():
            if k == title:
                title_list.append(v)
    return title_list

In [9]:
df["Title"] = group_titles(df, "Title", title_dictionary)          

In [10]:
df_test["Title"] = group_titles(df_test, "Title", title_dictionary)

# Exercise BONUS:
Try to extract some information from the feature **Ticket**

In [11]:
#print(df["Ticket"].nunique())
print(len(df["Ticket"]))
ticket_numeric = []
ticket_alpha = []
for ticket in df["Ticket"]:
    if ticket.isnumeric():
        ticket_numeric.append(ticket)
    else:
        ticket_alpha.append(ticket)
        
print(len(ticket_numeric))
print(ticket_numeric[:10])
print(len(ticket_alpha))
print(ticket_alpha[:10])

891
661
['113803', '373450', '330877', '17463', '349909', '347742', '237736', '113783', '347082', '350406']
230
['A/5 21171', 'PC 17599', 'STON/O2. 3101282', 'PP 9549', 'A/5. 2151', 'PC 17601', 'PC 17569', 'C.A. 24579', 'PC 17604', 'A./5. 2152']


# Exercise BONUS:
Try to extract some information from the feature **Cabin**

In [12]:
def cabin_letter(dataframe, column):
    dataframe[column] = dataframe[column].fillna("W")
    dataframe.loc[dataframe[column].notnull(), 'Cabin Letter'] = dataframe[column].str[0]

    
cabin_letter(df, "Cabin")        
cabin_letter(df_test, "Cabin")


In [13]:
import re

def cabin_amount(dataframe, column):
    cabin_amount_list = []
    for cabin in dataframe[column]:
        if cabin == "W":
            cabin_no = 0
        else:
            cabin_split = re.findall('[A-Z][^A-Z]*', cabin)
            cabin_no = len(cabin_split)
        cabin_amount_list.append(cabin_no)
    return cabin_amount_list
    
    
df["Cabin Amount"] = cabin_amount(df, "Cabin")
df_test["Cabin Amount"] = cabin_amount(df_test, "Cabin")

In [14]:
df.sample(n=10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Cabin Letter,Cabin Amount
PassengerId,,,,,,,,,,,,,,
285,0,1,"Smith, Mr. Richard William",male,NaN,0,0,113056,26.0000,A19,S,Mr,A,1
396,0,3,"Johansson, Mr. Erik",male,22.0,0,0,350052,7.7958,W,S,Mr,W,0
552,0,2,"Sharp, Mr. Percival James R",male,27.0,0,0,244358,26.0000,W,S,Mr,W,0
600,1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,PC 17485,56.9292,A20,C,Royalty,A,1
87,0,3,"Ford, Mr. William Neal",male,16.0,1,3,W./C. 6608,34.3750,W,S,Mr,W,0
309,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.0000,W,C,Mr,W,0
118,0,2,"Turpin, Mr. William John Robert",male,29.0,1,0,11668,21.0000,W,S,Mr,W,0
433,1,2,"Louch, Mrs. Charles Alexander (Alice Adelaide ...",female,42.0,1,0,SC/AH 3085,26.0000,W,S,Mrs,W,0
52,0,3,"Nosworthy, Mr. Richard Cater",male,21.0,0,0,A/4. 39886,7.8000,W,S,Mr,W,0


In [15]:
df_test.sample(n=10)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Cabin Letter,Cabin Amount
PassengerId,,,,,,,,,,,,,
1231,3,"Betros, Master. Seman",male,NaN,0,0,2622,7.2292,W,C,Master,W,0
1259,3,"Riihivouri, Miss. Susanna Juhantytar Sanni""""",female,22.0,0,0,3101295,39.6875,W,S,Miss,W,0
1162,1,"McCaffry, Mr. Thomas Francis",male,46.0,0,0,13050,75.2417,C6,C,Mr,C,1
940,1,"Bucknell, Mrs. William Robert (Emma Eliza Ward)",female,60.0,0,0,11813,76.2917,D15,C,Mrs,D,1
1159,3,"Warren, Mr. Charles William",male,NaN,0,0,C.A. 49867,7.5500,W,S,Mr,W,0
1239,3,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,0,0,2688,7.2292,W,C,Mrs,W,0
977,3,"Khalil, Mr. Betros",male,NaN,1,0,2660,14.4542,W,C,Mr,W,0
1208,1,"Spencer, Mr. William Augustus",male,57.0,1,0,PC 17569,146.5208,B78,C,Mr,B,1
942,1,"Smith, Mr. Lucien Philip",male,24.0,1,0,13695,60.0000,C31,S,Mr,C,1


# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [16]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin', 'Cabin Letter', 'Cabin Amount']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin', 'Cabin Letter', 'Cabin Amount']) # # X_TEST DATA (NEW DATA)

In [17]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3:
Create a **ColumnTransformer for Tree Models**. Remember:
- Categorical: Some SimpleImputer -> Some Encoder
- Numerical: Some SimpleImputer -> NO Encoder

In [18]:
num_preprocessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', add_indicator=True)), # mean, median
])

cat_preporcessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(add_indicator=True,
                                                                strategy='constant'))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4
1. Complete the dictionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [19]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [20]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

set_config(display='diagram') # Useful for display the pipeline

In [29]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

#Display the fullpipeline of the LGBMClassifier
tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='constant'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5:
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [22]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,
    random_state=0
)

In [23]:
for model_name, model in tree_classifiers.items():
    
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    model.fit(x_train, y_train)
    
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    y_pred = model.predict(x_val)
    print("\nAccuracy of:", model_name, metrics.accuracy_score(y_val, y_pred)*100)
    print("Balanced accuracy of:", model_name, metrics.balanced_accuracy_score(y_val, y_pred)*100)


Accuracy of: Decision Tree 80.44692737430168
Balanced accuracy of: Decision Tree 78.4189723320158

Accuracy of: Extra Trees 75.97765363128491
Balanced accuracy of: Extra Trees 73.16205533596838

Accuracy of: Random Forest 79.3296089385475
Balanced accuracy of: Random Forest 76.96969696969697

Accuracy of: AdaBoost 82.12290502793296
Balanced accuracy of: AdaBoost 80.05270092226614

Accuracy of: Skl GBM 84.35754189944134
Balanced accuracy of: Skl GBM 81.87088274044795

Accuracy of: Skl HistGBM 81.56424581005587
Balanced accuracy of: Skl HistGBM 79.05797101449275
[18:00:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Accuracy of: XGBoost 81.56424581005587
Balanced accuracy of: XGBoost 79.05797101449275


/home/klarakhalo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Accuracy of: LightGBM 81.00558659217877
Balanced accuracy of: LightGBM 78.33333333333333
Learning rate set to 0.008911
0:	learn: 0.6876139	total: 56ms	remaining: 56s
1:	learn: 0.6810414	total: 67.5ms	remaining: 33.7s
2:	learn: 0.6761196	total: 71ms	remaining: 23.6s
3:	learn: 0.6704200	total: 80.5ms	remaining: 20s
4:	learn: 0.6650584	total: 82.2ms	remaining: 16.4s
5:	learn: 0.6598222	total: 85.4ms	remaining: 14.2s
6:	learn: 0.6546137	total: 87.7ms	remaining: 12.4s
7:	learn: 0.6490173	total: 90.1ms	remaining: 11.2s
8:	learn: 0.6443643	total: 93.3ms	remaining: 10.3s
9:	learn: 0.6393528	total: 98.1ms	remaining: 9.71s
10:	learn: 0.6335774	total: 105ms	remaining: 9.42s
11:	learn: 0.6278370	total: 107ms	remaining: 8.83s
12:	learn: 0.6233138	total: 111ms	remaining: 8.43s
13:	learn: 0.6184623	total: 114ms	remaining: 8.01s
14:	learn: 0.6139776	total: 119ms	remaining: 7.79s
15:	learn: 0.6102335	total: 123ms	remaining: 7.56s
16:	learn: 0.6063707	total: 126ms	remaining: 7.27s
17:	learn: 0.6016815	

190:	learn: 0.3720236	total: 534ms	remaining: 2.26s
191:	learn: 0.3717555	total: 538ms	remaining: 2.26s
192:	learn: 0.3715314	total: 541ms	remaining: 2.26s
193:	learn: 0.3712961	total: 543ms	remaining: 2.26s
194:	learn: 0.3708781	total: 545ms	remaining: 2.25s
195:	learn: 0.3703410	total: 547ms	remaining: 2.24s
196:	learn: 0.3700830	total: 549ms	remaining: 2.24s
197:	learn: 0.3698719	total: 551ms	remaining: 2.23s
198:	learn: 0.3694921	total: 556ms	remaining: 2.24s
199:	learn: 0.3690914	total: 560ms	remaining: 2.24s
200:	learn: 0.3687695	total: 561ms	remaining: 2.23s
201:	learn: 0.3685142	total: 562ms	remaining: 2.22s
202:	learn: 0.3681724	total: 563ms	remaining: 2.21s
203:	learn: 0.3677637	total: 564ms	remaining: 2.2s
204:	learn: 0.3675429	total: 566ms	remaining: 2.19s
205:	learn: 0.3673335	total: 568ms	remaining: 2.19s
206:	learn: 0.3669617	total: 572ms	remaining: 2.19s
207:	learn: 0.3665220	total: 574ms	remaining: 2.19s
208:	learn: 0.3663254	total: 578ms	remaining: 2.19s
209:	learn: 0

418:	learn: 0.3262555	total: 904ms	remaining: 1.25s
419:	learn: 0.3261852	total: 906ms	remaining: 1.25s
420:	learn: 0.3261164	total: 907ms	remaining: 1.25s
421:	learn: 0.3258918	total: 910ms	remaining: 1.25s
422:	learn: 0.3256505	total: 912ms	remaining: 1.24s
423:	learn: 0.3254976	total: 914ms	remaining: 1.24s
424:	learn: 0.3253607	total: 917ms	remaining: 1.24s
425:	learn: 0.3253181	total: 919ms	remaining: 1.24s
426:	learn: 0.3252152	total: 920ms	remaining: 1.24s
427:	learn: 0.3251104	total: 922ms	remaining: 1.23s
428:	learn: 0.3249231	total: 924ms	remaining: 1.23s
429:	learn: 0.3249205	total: 926ms	remaining: 1.23s
430:	learn: 0.3246579	total: 927ms	remaining: 1.22s
431:	learn: 0.3245263	total: 930ms	remaining: 1.22s
432:	learn: 0.3244108	total: 931ms	remaining: 1.22s
433:	learn: 0.3243372	total: 932ms	remaining: 1.22s
434:	learn: 0.3242506	total: 934ms	remaining: 1.21s
435:	learn: 0.3241123	total: 935ms	remaining: 1.21s
436:	learn: 0.3240608	total: 936ms	remaining: 1.21s
437:	learn: 

666:	learn: 0.2990545	total: 1.27s	remaining: 634ms
667:	learn: 0.2989215	total: 1.27s	remaining: 632ms
668:	learn: 0.2988482	total: 1.27s	remaining: 630ms
669:	learn: 0.2988005	total: 1.27s	remaining: 628ms
670:	learn: 0.2987143	total: 1.28s	remaining: 626ms
671:	learn: 0.2985855	total: 1.28s	remaining: 625ms
672:	learn: 0.2985626	total: 1.28s	remaining: 622ms
673:	learn: 0.2985093	total: 1.28s	remaining: 620ms
674:	learn: 0.2984675	total: 1.28s	remaining: 618ms
675:	learn: 0.2984261	total: 1.28s	remaining: 616ms
676:	learn: 0.2982931	total: 1.29s	remaining: 614ms
677:	learn: 0.2981855	total: 1.29s	remaining: 611ms
678:	learn: 0.2981134	total: 1.29s	remaining: 609ms
679:	learn: 0.2980354	total: 1.29s	remaining: 607ms
680:	learn: 0.2979973	total: 1.29s	remaining: 606ms
681:	learn: 0.2978553	total: 1.29s	remaining: 604ms
682:	learn: 0.2977959	total: 1.3s	remaining: 603ms
683:	learn: 0.2977420	total: 1.3s	remaining: 602ms
684:	learn: 0.2976640	total: 1.3s	remaining: 600ms
685:	learn: 0.2

897:	learn: 0.2743172	total: 1.64s	remaining: 187ms
898:	learn: 0.2743058	total: 1.65s	remaining: 185ms
899:	learn: 0.2741355	total: 1.65s	remaining: 183ms
900:	learn: 0.2740243	total: 1.65s	remaining: 181ms
901:	learn: 0.2739791	total: 1.65s	remaining: 179ms
902:	learn: 0.2739390	total: 1.65s	remaining: 177ms
903:	learn: 0.2738303	total: 1.65s	remaining: 175ms
904:	learn: 0.2737434	total: 1.65s	remaining: 174ms
905:	learn: 0.2736200	total: 1.65s	remaining: 172ms
906:	learn: 0.2736023	total: 1.66s	remaining: 170ms
907:	learn: 0.2736008	total: 1.66s	remaining: 168ms
908:	learn: 0.2734797	total: 1.66s	remaining: 166ms
909:	learn: 0.2733679	total: 1.66s	remaining: 164ms
910:	learn: 0.2732335	total: 1.66s	remaining: 162ms
911:	learn: 0.2730978	total: 1.66s	remaining: 161ms
912:	learn: 0.2730474	total: 1.67s	remaining: 159ms
913:	learn: 0.2730081	total: 1.67s	remaining: 157ms
914:	learn: 0.2728113	total: 1.67s	remaining: 155ms
915:	learn: 0.2727814	total: 1.67s	remaining: 153ms
916:	learn: 

# Exercise 6:
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [24]:
from sklearn.model_selection import cross_val_predict

In [25]:
skf = model_selection.StratifiedKFold(
    n_splits=10, shuffle=True, random_state=0)

for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    pred = cross_val_predict(model, x, y, cv=skf)
    #model.fit(x_train, y_train)
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    #y_pred = model.predict(skf_split)
    print("\nAccuracy of:", model_name, metrics.accuracy_score(y, pred)*100)
    print("Balanced accuracy of:", model_name, metrics.balanced_accuracy_score(y, pred)*100)


Accuracy of: Decision Tree 79.91021324354658
Balanced accuracy of: Decision Tree 78.68133448375036

Accuracy of: Extra Trees 80.69584736251403
Balanced accuracy of: Extra Trees 79.0983606557377

Accuracy of: Random Forest 81.59371492704825
Balanced accuracy of: Random Forest 80.15770300067108

Accuracy of: AdaBoost 81.9304152637486
Balanced accuracy of: AdaBoost 80.87192023775285

Accuracy of: Skl GBM 83.61391694725027
Balanced accuracy of: Skl GBM 81.63167481545393

Accuracy of: Skl HistGBM 83.05274971941638
Balanced accuracy of: Skl HistGBM 81.39679800594382
[18:00:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/klarakhalo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/klarakhalo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:00:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/klarakhalo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:00:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

994:	learn: 0.2658029	total: 1.51s	remaining: 7.57ms
995:	learn: 0.2657096	total: 1.51s	remaining: 6.06ms
996:	learn: 0.2656380	total: 1.51s	remaining: 4.54ms
997:	learn: 0.2654741	total: 1.51s	remaining: 3.03ms
998:	learn: 0.2654096	total: 1.51s	remaining: 1.51ms
999:	learn: 0.2652005	total: 1.51s	remaining: 0us
Learning rate set to 0.009376
0:	learn: 0.6874696	total: 908us	remaining: 908ms
1:	learn: 0.6806797	total: 3.29ms	remaining: 1.64s
2:	learn: 0.6752584	total: 4.58ms	remaining: 1.52s
3:	learn: 0.6693338	total: 5.81ms	remaining: 1.45s
4:	learn: 0.6628464	total: 7ms	remaining: 1.39s
5:	learn: 0.6572686	total: 7.87ms	remaining: 1.3s
6:	learn: 0.6510599	total: 9.03ms	remaining: 1.28s
7:	learn: 0.6459923	total: 10.1ms	remaining: 1.26s
8:	learn: 0.6408488	total: 11.2ms	remaining: 1.24s
9:	learn: 0.6351044	total: 12.6ms	remaining: 1.24s
10:	learn: 0.6292293	total: 14ms	remaining: 1.26s
11:	learn: 0.6235942	total: 17.5ms	remaining: 1.44s
12:	learn: 0.6190081	total: 19.7ms	remaining: 1.

203:	learn: 0.3745952	total: 296ms	remaining: 1.16s
204:	learn: 0.3742790	total: 302ms	remaining: 1.17s
205:	learn: 0.3737986	total: 305ms	remaining: 1.17s
206:	learn: 0.3737197	total: 306ms	remaining: 1.17s
207:	learn: 0.3736554	total: 307ms	remaining: 1.17s
208:	learn: 0.3735138	total: 308ms	remaining: 1.17s
209:	learn: 0.3732362	total: 310ms	remaining: 1.17s
210:	learn: 0.3727963	total: 312ms	remaining: 1.17s
211:	learn: 0.3722923	total: 314ms	remaining: 1.17s
212:	learn: 0.3720207	total: 317ms	remaining: 1.17s
213:	learn: 0.3717150	total: 319ms	remaining: 1.17s
214:	learn: 0.3714495	total: 321ms	remaining: 1.17s
215:	learn: 0.3712212	total: 324ms	remaining: 1.18s
216:	learn: 0.3708535	total: 327ms	remaining: 1.18s
217:	learn: 0.3705010	total: 330ms	remaining: 1.18s
218:	learn: 0.3704711	total: 331ms	remaining: 1.18s
219:	learn: 0.3700371	total: 333ms	remaining: 1.18s
220:	learn: 0.3696858	total: 334ms	remaining: 1.18s
221:	learn: 0.3692753	total: 337ms	remaining: 1.18s
222:	learn: 

426:	learn: 0.3336065	total: 666ms	remaining: 893ms
427:	learn: 0.3334105	total: 670ms	remaining: 895ms
428:	learn: 0.3333026	total: 674ms	remaining: 897ms
429:	learn: 0.3331844	total: 675ms	remaining: 895ms
430:	learn: 0.3330354	total: 677ms	remaining: 894ms
431:	learn: 0.3330280	total: 678ms	remaining: 892ms
432:	learn: 0.3329506	total: 680ms	remaining: 891ms
433:	learn: 0.3328538	total: 682ms	remaining: 889ms
434:	learn: 0.3327062	total: 683ms	remaining: 887ms
435:	learn: 0.3326393	total: 684ms	remaining: 885ms
436:	learn: 0.3324937	total: 686ms	remaining: 883ms
437:	learn: 0.3323827	total: 687ms	remaining: 882ms
438:	learn: 0.3323787	total: 688ms	remaining: 879ms
439:	learn: 0.3323366	total: 689ms	remaining: 877ms
440:	learn: 0.3322484	total: 691ms	remaining: 875ms
441:	learn: 0.3322353	total: 691ms	remaining: 873ms
442:	learn: 0.3319738	total: 693ms	remaining: 871ms
443:	learn: 0.3318341	total: 695ms	remaining: 870ms
444:	learn: 0.3316945	total: 697ms	remaining: 869ms
445:	learn: 

657:	learn: 0.3067675	total: 1.03s	remaining: 537ms
658:	learn: 0.3066737	total: 1.03s	remaining: 535ms
659:	learn: 0.3065003	total: 1.04s	remaining: 535ms
660:	learn: 0.3064072	total: 1.04s	remaining: 534ms
661:	learn: 0.3063163	total: 1.04s	remaining: 532ms
662:	learn: 0.3062580	total: 1.04s	remaining: 531ms
663:	learn: 0.3062183	total: 1.05s	remaining: 530ms
664:	learn: 0.3061590	total: 1.05s	remaining: 528ms
665:	learn: 0.3060855	total: 1.05s	remaining: 526ms
666:	learn: 0.3059940	total: 1.05s	remaining: 525ms
667:	learn: 0.3059054	total: 1.05s	remaining: 523ms
668:	learn: 0.3058106	total: 1.05s	remaining: 522ms
669:	learn: 0.3056831	total: 1.05s	remaining: 520ms
670:	learn: 0.3056406	total: 1.06s	remaining: 518ms
671:	learn: 0.3055844	total: 1.06s	remaining: 517ms
672:	learn: 0.3054960	total: 1.06s	remaining: 515ms
673:	learn: 0.3053701	total: 1.06s	remaining: 513ms
674:	learn: 0.3053543	total: 1.06s	remaining: 512ms
675:	learn: 0.3052230	total: 1.06s	remaining: 510ms
676:	learn: 

870:	learn: 0.2850745	total: 1.4s	remaining: 207ms
871:	learn: 0.2850255	total: 1.4s	remaining: 206ms
872:	learn: 0.2848203	total: 1.4s	remaining: 204ms
873:	learn: 0.2847458	total: 1.4s	remaining: 202ms
874:	learn: 0.2846885	total: 1.41s	remaining: 201ms
875:	learn: 0.2845491	total: 1.41s	remaining: 199ms
876:	learn: 0.2845233	total: 1.41s	remaining: 198ms
877:	learn: 0.2843391	total: 1.41s	remaining: 196ms
878:	learn: 0.2841538	total: 1.41s	remaining: 195ms
879:	learn: 0.2841130	total: 1.42s	remaining: 194ms
880:	learn: 0.2839283	total: 1.43s	remaining: 193ms
881:	learn: 0.2838830	total: 1.43s	remaining: 191ms
882:	learn: 0.2838440	total: 1.43s	remaining: 190ms
883:	learn: 0.2837867	total: 1.43s	remaining: 188ms
884:	learn: 0.2837605	total: 1.43s	remaining: 186ms
885:	learn: 0.2836608	total: 1.44s	remaining: 185ms
886:	learn: 0.2835764	total: 1.44s	remaining: 184ms
887:	learn: 0.2834284	total: 1.44s	remaining: 182ms
888:	learn: 0.2833924	total: 1.45s	remaining: 181ms
889:	learn: 0.28

# Exercise 7
Train **with all data** the best model

In [30]:
best_model = tree_classifiers["Skl GBM"]
# Train with all data your best model
best_model.fit(x,y)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='constant'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 8
With your best model, generate the predicitions for test data (x_test)

In [31]:
test_pred = best_model.predict(x_test)
test_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

# Exercise 9

Submit to kaggle using the kaggle API. And send us your score. You can try to improve it.

In [32]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0


In [33]:
sub.to_csv("submission.csv")

### My score on my kaggle submission was 0.77751

# Exercise BONUS

Knowing how to export your models is very important for putting models in production. Try to
- Export and Load the ColumTransformer in pickle
- Export and Load the ColumTransformer in joblib
- Export and load the Pipeline